In [14]:
import tushare as ts 
import pandas as pd
import numpy as np 
from datetime import datetime,timedelta
from abc import ABCMeta, abstractmethod

class Stock_Select():
    def __init__(self,index_code,total_mv,share_ratio,date):
        self.pro = ts.pro_api()
        self.stock_list = []
        self.total_mv = total_mv
        self.share_ratio = share_ratio 
        self.index_code = index_code
        self.fina = pd.DataFrame()
        self.get_sw_codes()
        self.final_codes = []
        self.date = date
        

    @abstractmethod
    def get_sw_codes(self):
        raise NotImplemntedError('Should implement generate signal method!')
        
    def get_code(self):
        #date = (datetime.now() + timedelta(days=-3)).strftime('%Y%m%d') #当前日期的前一天，转为字符串 
        data = pd.DataFrame()   
        for i in self.stock_list:
            df_db= self.pro.query('daily_basic', ts_code=i, trade_date=self.date,fields='ts_code,trade_date,\
            turnover_rate,volume_ratio,pe,pb,total_share,free_share,total_mv')
            data =data.append(df_db)
        df_db =   data.copy() 
       
        df_db['share_ratio'] = df_db.free_share / df_db.total_share * 100 #计算自由流通股占比
        df_db['total_mv'] = df_db.total_mv  / 10000 #把总市值单位转为亿元
        filt = (df_db['total_mv'] > 50 ) & (df_db['share_ratio'] > 20)  #总市值 > 100亿,流通股占比大于20%
        df_db =  df_db[filt] 
        c_1 = set(df_db.ts_code.to_list()) #取set
        
        df = self.pro.query('stock_basic', exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
        year = (datetime.now() +timedelta(days=-365*3)).strftime('%Y%m%d') #上市时间为三年前的今天
        #保留上市时间大于三年个股数据
        df=df[df.list_date<year]
        #排除银行、保险、多元金融公司
        df=df[-df.industry.isin(['银行','保险','多元金融'])]
        #排除st和*ST股
        df=df[-df.name.str.startswith(('ST'))]
        df=df[-df.name.str.startswith(('*'))] 
        c_2 = set(df.ts_code.to_list())

        c = c_1 & c_2 #求交集
        df = df[df.ts_code.isin(c)]
        code=df.ts_code.values
        name=df.name.values
#         print(dict(zip(name,code)))
        return dict(zip(name,code))
    def get_indicators(self,code):
        #获取当前时间，计算当前和过去四年时间
        t0=datetime.now()
        t1=datetime(t0.year-4,t0.month,t0.day)
        end=t0.strftime('%Y%m%d')
        start=t1.strftime('%Y%m%d')
        
         #财务比率
        fields='ann_date,end_date,tr_yoy,op_yoy,\
             grossprofit_margin,expense_of_sales,inv_turn,eps,\
             ocfps,roe_yearly,roa2_yearly,netprofit_yoy'
        fina = (self.pro.fina_indicator(ts_code=code,start_date=start, end_date=end,fields=fields)
               .drop_duplicates(subset=['ann_date','end_date'],keep='first'))
        fina.set_index('end_date',inplace = True)
        fina=fina.sort_index()
        fina['eps_yearly'] = fina['eps'] * [1,12/3,12/6,12/9,1,12/3,12/6,12/9,1,12/3,12/6,12/9,1,12/3,12/6,12/9]
        fina['inv_turn_yearly'] = fina['inv_turn'] * [1,12/3,12/6,12/9,1,12/3,12/6,12/9,1,12/3,12/6,12/9,1,12/3,12/6,12/9]
        fina['ocfps_yearly'] = fina['ocfps'] * [1,12/3,12/6,12/9,1,12/3,12/6,12/9,1,12/3,12/6,12/9,1,12/3,12/6,12/9]
        fina['expense_of_sales_yearly'] = fina['expense_of_sales'] * [1,12/3,12/6,12/9,1,12/3,12/6,12/9,1,12/3,12/6,12/9,1,12/3,12/6,12/9]
        '''
         {'年化每股收益':'eps_yearly','年化存货周转率':'inv_turn_yearly','年化每股经营性现金流量':'ocfps_yearly',\
         '毛利率':'grossprofit_margin','年化期间费用率':'expense_of_sales_yearly',\
          '年化净资产收益率':'roe_yearly','年化总资产报酬率':'roa2_yearly','营业利润同比增长率':'op_yoy',\
          '销售净利润率同比增长':'netprofit_yoy', '营业收入同比增长率':'tr_yoy'      
         }
        '''
        cols = ['eps_yearly','inv_turn_yearly','ocfps_yearly','grossprofit_margin','expense_of_sales_yearly','roe_yearly','roa2_yearly',\
                'op_yoy','tr_yoy']
        fina = fina[cols]
        fina = fina.fillna(method='ffill') #向下填充  
        self.fina = fina.copy()
#         print(self.fina)
        return self.fina
    
        #存在缺失值或者负值（市盈率）情况下评分直接为0
        #营业收入增长率打分（0-10）
    def cal_tryoy(self,y):
        '''y是营业收入增长率'''
        try:
            return 5+ min(round(y-10),5) if y>=10 else 5+ max(round(y-10),-5)
        except:
            return 0

    #营业利润增长率打分(0-10)
    def cal_opyoy(self,y):
        '''y是营业利润增长率'''
        try:
            return 5+ min(round((y-20)/2),5) if y>=20 else 5+ max(round((y-20)/2),-5)
        except:
            return 0

    #毛利率打分
    def cal_gpm(self,y):
        '''y是最近季度毛利率-前三季度平均毛利率'''
        try:
            return 5+min(round(y)/0.5,5) if y>0 else max(round(y)/0.5,-5)+5
        except:
            return 0
    #期间费用率打分
    def cal_exp(self,y):
        '''y是最近季度期间费用率-前三季度平均期间费用率'''
        try:
            return 5+min(round(y)/0.5,5) if y>0 else max(round(y)/0.5,-5)+5
        except:
            return 0
    #存货周转率打分
    def cal_inv(self,y):
        '''y是（最近季度存货周转率-前三季度平均存货周转率）/前三季度平均存货周转率*100'''
        try:
            return 5+min(round(y/2),5) if y>0 else max(round(y/2),-5)+5
        except:
            return 0
    #每股经营性现金流打分
    def cal_ocfp(self,y):
        '''y是（最近三季度每股经营性现金流之和-最近三季度每股收益之和）/最近三季度每股收益之和*100'''
        try:
            return 5+min(round(y/4),5) if y>0 else max(round(y/4),-5)+5
        except:
            return 0
    #净资产收益率打分
    def cal_roe(self,y):
        '''y是年化净资产收益率'''
        try:
            return 5+ min(round(y-15),5) if y>=15 else 5+ max(round(y-15),-5)
        except:
            return 0
    #总资产报酬率打分
    def cal_roa(self,y):
        '''y是最近季度年化总资产报酬率'''
        try:
            return min(round((y-5)/0.5),10) if y>=5 else max(round(y-5),0)
        except:
            return 0
    #计算财务指标得分
    def indicator_score(self,code):
        data = self.get_indicators(code)
        ''''(1)营业收入增长率打分'''
        data['营收得分'] = data['tr_yoy'].apply(self.cal_tryoy)
        '''(2)营业利润增长率打分'''
        data['利润得分'] = data['op_yoy'].apply(self.cal_opyoy)
        '''(3)毛利率打分'''
        #计算最近季度毛利率-前三季度平均毛利率
        data['gpm']=data['grossprofit_margin']-data['grossprofit_margin'].rolling(3).mean()
        data['毛利率得分']=data['gpm'].apply(self.cal_gpm)
        '''(4)期间费用率打分'''
        #最近季度期间费用率-前三季度平均期间费用率
        data['exp']=data['expense_of_sales_yearly']-data['expense_of_sales_yearly'].rolling(3).mean()
        data['费用得分']=data['exp'].apply(self.cal_exp)
        '''(5)周转率打分'''
        #（最近季度存货周转率-前三季度平均存货周转率）/前三季度平均存货周转率*100
        data['inv']=(data['inv_turn_yearly']-data['inv_turn_yearly'].rolling(3).mean())*100/data['inv_turn_yearly'].rolling(3).mean()
        data['周转得分']=data['inv'].apply(self.cal_inv)
        '''(6)每股经营现金流打分'''
        #（最近三季度每股经营性现金流之和-最近三季度每股收益之和）/最近三季度每股收益之和*100
        data['ocf']=(data['ocfps_yearly'].rolling(3).sum()-data['eps_yearly'].rolling(3).sum())*100/data['eps_yearly'].rolling(3).sum()
        data['现金得分']=data['ocf'].apply(self.cal_ocfp)
        '''(7)净资产收益率打分'''
        data['净资产得分']=data['roe_yearly'].apply(self.cal_roe)
        '''(8)总资产收益率打分'''
        data['总资产得分']=data['roa2_yearly'].apply(self.cal_roa)
        data = data.iloc[2:]
        #计算总得分
        data['总分']=data[['营收得分','利润得分','毛利率得分','费用得分','周转得分','现金得分','净资产得分','总资产得分']].sum(axis=1)
#         print(data)
        return data[['营收得分','利润得分','毛利率得分','费用得分','周转得分','现金得分','净资产得分','总资产得分','总分']]
    # 计算所有股票财务指标总分
    def get_all_score(self):
        dff=pd.DataFrame()
        for name,code in self.get_code().items():
            try:
                dff[name]= self.indicator_score(code)['总分'].copy()               
            except:
                continue  
#         print(dff)
        
        return dff.T.sort_values(dff.T.columns[-1],ascending=False)
       
    
class Stock(Stock_Select):
    def __init__(self,index_code,total_mv,share_ratio,date):
        super(Stock,self).__init__(index_code,total_mv,share_ratio,date)
        
    def get_sw_codes(self):
        """
        查询申万指数成分股
        """
        df_stock= self.pro.index_member(index_code=self.index_code)
        if df_stock.empty:
            print('查询不到成分股')
        else:
            self.stock_list = df_stock.con_code.to_list()        
            return self.stock_list
        """
        查询
        {'上证50': '000016.SH','沪深300': '000300.SH','中证500': '000905.SH','中证800': '000906.SH','沪深300高贝塔': '000828.SH',\
         '沪深300低贝塔(SH)': '000829.SH','中证500高贝塔(SH)': '000830.SH','中证500低贝塔(SH)': '000831.SH'        
        }
        """
#         df_stock = self.pro.index_weight(index_code=self.index_code, start_date='20210901', end_date='20211001')
#         self.stock_list =  df_stock.con_code.unique().tolist()
#         if len(self.stock_list):
#             return self.stock_list
#         else:
#             print('查询不到成分股')
    def eval(self):
        starttime = datetime.now()
        df = self.get_all_score()
        df['均值'] = df.iloc[:,-20:].mean(axis=1)  #对总分求均值
        self.final_codes = df.sort_values(df.columns[-1],ascending=False).iloc[0:25,-1].index.to_list()       
#         print(round(df.sort_values(df.columns[-1],ascending=False).iloc[0:25,-1],1)) # 获取平均总分排名前15个股
        endtime = datetime.now()
        print('打分完成，耗时:'+str((endtime - starttime).seconds)+' seconds') 
        return round(df.sort_values(df.columns[-1],ascending=False).iloc[0:25,-1],1)
    def stock_price_eval(self,date):
        data = self.pro.query('stock_basic', exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
        frame = data[data['name'].isin(self.final_codes)]
#         print(self.final_codes)
#         print(frame[frame['name'].isin(self.final_codes)])
        codes = frame.ts_code.values.tolist()
        names = frame.name.values.tolist()
        stock_dict = dict(zip(names,codes))
        stock_price_eval = dict()
        self.date = date 
        
        for name,code in stock_dict.items():
#             print(name,code)
            pe_ttm,close,pb,ps = self.get_daily_basic(code,self.date)
#             print(name,pe_ttm,close)
            eps_score,bps_score,ps_score =  self.get_fina_indicator(code)
#             print(eps_score)
            if pd.isna(pe_ttm):
                if pd.isna(pb):
                    eval_price =  round(ps_score * ps,3)
                else:
                    eval_price =  round(bps_score * pb,3)
            else:
                eval_price = round(eps_score * pe_ttm,3) 
            stock_price_eval[name] = [eval_price,close]        
        
        return stock_price_eval
                               
    
    def get_daily_basic(self,ts_code,date):
        df = self.pro.query('daily_basic', ts_code=ts_code, trade_date=date,fields='ts_code,trade_date,close,pe_ttm,pb,ps')#         
        return df['pe_ttm'][0],df['close'][0],df['pb'][0],df['ps'][0]
    def get_fina_indicator(self,ts_code):
        df = self.pro.query('fina_indicator', ts_code=ts_code, start_date='20210101', end_date='20211125')
        cols = ['ts_code','ann_date','end_date','eps','bps','revenue_ps']
        df = df.loc[:,cols].copy()
        df.drop_duplicates(inplace = True)
        
        eps_score = (df['eps'][-3:] * [12/9,12/6,12/3]).mean()
        bps_score = (df['bps'][-3:] * [1,1,1]).mean()
        ps_score = (df['revenue_ps'][-3:] * [12/9,12/6,12/3]).mean()

        return eps_score,bps_score,ps_score
        
        
        

if __name__ == '__main__':
    date = (datetime.now() + timedelta(days=-2)).strftime('%Y%m%d') #当前日期的前一天，转为字符串
#     date = datetime.now().strftime('%Y%m%d')  # 今天日期
    s = Stock('801151.SI',50,20,date)
    
    frame = s.eval() 
    df = s.stock_price_eval(date)
    df = pd.DataFrame(df).T
    df.columns = ['估值','收盘']
    
    df = pd.merge(frame,df,how='left',left_index=True,right_index=True)
    df['上涨空间'] = round((df['估值'] - df['收盘']) /  df['收盘'] * 100,2)
    
    print(df)

     
   

C:\Users\rufus\.conda\envs\rufus\lib\site-packages\ipykernel_launcher.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


打分完成，耗时:20 seconds
        均值       估值      收盘   上涨空间
药石科技  53.5  175.911  148.00  18.86
普洛药业  52.7   38.599   36.72   5.12
健友股份  51.3   52.462   41.17  27.43
普利制药  49.9   48.199   53.23  -9.45
恩华药业  47.3   14.328   14.80  -3.19
金达威   47.1   38.802   30.90  25.57
凯莱英   47.1  450.748  470.00  -4.10
司太立   46.6   72.056   65.11  10.67
兴齐眼药  46.1  112.839  128.00 -11.84
贝达药业  45.6   93.145   82.11  13.44
恒瑞医药  45.2   49.446   51.26  -3.54
奥翔药业  44.9   46.965   35.58  32.00
新和成   43.4   34.863   28.28  23.28
东阳光   43.4    4.179    9.89 -57.75
东诚药业  42.7   13.789   15.24  -9.52
花园生物  42.5   17.235   13.62  26.54
一品红   42.4   38.881   33.51  16.03
健康元   42.1   13.733   12.13  13.22
华特达因  42.1   32.693   28.28  15.60
仙琚制药  41.1   10.460   11.86 -11.80
丽珠集团  40.6   26.489   37.37 -29.12
华润双鹤  40.2   13.789   12.00  14.91
天宇股份  39.8   73.968   41.04  80.23
博腾股份  39.5   88.830   95.00  -6.49
富祥药业  39.4   24.509   16.43  49.17


In [13]:
import tushare as ts 
pro = ts.pro_api()

df = pro.sw_daily(start_date='20211125', end_date='20211125')
df[df['name'].str.contains('化学制药')]

,ts_code,trade_date,name,open,low,high,close,change,pct_change,vol,amount,pe,pb
63,801151.SI,20211125,化学制药,11916.8,11800.88,12192.75,12046.77,126.94,1.06,123391.0,2091314.0,40.45,3.77


In [56]:
"""
上交所指数-规模指数
{'上证380': '000009.SH', '上证180': '000010.SH', '上证50': '000016.SH', '超大盘': '000043.SH', '上证中盘': '000044.SH', '上证小盘': '000045.SH',
 '上证中小': '000046.SH', '上证全指': '000047.SH', '上证市值百强': '000155.SH', '沪深300': '000300.SH', 'test': '000680.SH', '科创50': '000688.SH',
 '科创50(全)': '000688CNY01.SH', '科创50(净)': '000688CNY02.SH', '中证1000': '000852.SH', '中证流通(SH)': '000902.SH', '中证100': '000903.SH',
 '中证200': '000904.SH', '中证500': '000905.SH', '中证800': '000906.SH', '中证700(SH)': '000907.SH', '中证超大(SH)': '000980.SH', '中证全指(SH)': '000985.SH',
 '两岸三地': '000999.SH', '中证800(全)USD': '921459.SH', '中证1000(全)USD': '921460.SH', '中证800USD': '931459.SH', '中证1000USD': '931460.SH',
 '上证380全收益': 'h00009.SH', '上证180全收益': 'h00010.SH', '上证50全收益': 'h00016.SH', '超大盘全收益': 'h00043.SH', '上证中盘全收益': 'h00044.SH',
 '上证小盘全收益': 'h00045.SH', '上证中小全收益': 'h00046.SH', '上证全指全收益': 'h00047.SH', '上证100全收益': 'h00132.SH', '上证150全收益': 'h00133.SH',
 '上证市值百强全收益': 'h00155.SH', '中证1000全收益': 'h00852.SH', '上证380净收益': 'n00009.SH', '上证180净收益': 'n00010.SH', '上证50净收益': 'n00016.SH',
 '上证超级大盘净收益': 'n00043.SH', '上证中盘净收益': 'n00044.SH', '上证小盘净收益': 'n00045.SH', '上证中小净收益': 'n00046.SH', '上证全指净收益': 'n00047.SH',
 '上证100净收益': 'n00132.SH', '上证150净收益': 'n00133.SH', '市值百强净收益': 'n00155.SH', '中证800(净)USD': 'n21459.SH', '中证1000(净)USD': 'n21460.SH',
 'S812T指数': 'S812T.SH'}
 
"""
pro = ts.pro_api()
df = pro.index_basic(market='SSE')
df
code = df[df['category']=='策略指数'].ts_code.values
name = df[df['category']=='策略指数'].name.values
dict(zip(name,code))
"""
{'50等权': '000050.SH', '180等权': '000051.SH', '50基本': '000052.SH', '180基本': '000053.SH', '能源等权': '000070.SH', '材料等权': '000071.SH',
 '工业等权': '000072.SH', '可选等权': '000073.SH', '消费等权': '000074.SH', '医药等权': '000075.SH', '金融等权': '000076.SH', '信息等权': '000077.SH',
 '电信等权': '000078.SH', '公用等权': '000079.SH', '180分层': '000093.SH', '上证F200': '000098.SH', '上证F300': '000099.SH', '上证F500': '000100.SH',
 '380等权': '000115.SH', '100等权': '000124.SH', '380基本': '000128.SH', '180波动': '000129.SH', '380波动': '000130.SH', '上证180高贝塔': '000135.SH',
 '上证180低贝塔': '000136.SH', '上证380高贝塔': '000137.SH', '上证380低贝塔': '000138.SH', '50AH优选': '000170.SH', '300波动(SH)': '000803.SH',
 '500波动(SH)': '000804.SH', '沪深300高贝塔': '000828.SH', '沪深300低贝塔(SH)': '000829.SH', '中证500高贝塔(SH)': '000830.SH', '中证500低贝塔(SH)': '000831.SH',
 '中证800等权': '000842.SH', '沪深300动态(SH)': '000843.SH', '沪深300稳定(SH)': '000844.SH', '百发100': '000851.SH', '基本200(SH)': '000965.SH',
 '基本400': '000966.SH', '基本600(SH)': '000967.SH', '等权90': '000971.SH', '300分层(SH)': '000981.SH', '500等权': '000982.SH', '300等权': '000984.SH',
 '50AH优选(HKD)': '950091.SH', '上证180优选': '950092.SH', '上证180优选(HKD)': '950093.SH', '上证增持': '950094.SH', '上证定增': '950095.SH', '180SNLV': '950098.SH',
 '380SNLV': '950099.SH', '50等权全收益': 'h00050.SH', '180等权全收益': 'h00051.SH', '50基本全收益': 'h00052.SH', '180基本全收益': 'h00053.SH', '能源等权全收益': 'h00070.SH',
 '材料等权全收益': 'h00071.SH', '工业等权全收益': 'h00072.SH', '可选等权全收益': 'h00073.SH', '消费等权全收益': 'h00074.SH', '医药等权全收益': 'h00075.SH',
 '金融等权全收益': 'h00076.SH', '信息等权全收益': 'h00077.SH', '电信等权全收益': 'h00078.SH', '公用等权全收益': 'h00079.SH', '180分层全收益': 'h00093.SH',
 '上证F200全收益': 'h00098.SH', '上证F300全收益': 'h00099.SH', '上证F500全收益': 'h00100.SH', '380等权全收益': 'h00115.SH', '380基本全收益': 'h00128.SH',
 '180波动全收益': 'h00129.SH', '380波动全收益': 'h00130.SH', '上证180高贝塔全收益': 'h00135.SH', '上证180低贝塔全收益': 'h00136.SH',
 '上证380高贝塔全收益': 'h00137.SH', '上证380低贝塔全收益': 'h00138.SH', '上证50优选全收益': 'h40090.SH', '上证50优选(HKD)全收益': 'h40091.SH',
 '上证180优选全收益': 'h40092.SH', '上证180优选(HKD)全收益': 'h40093.SH', '180SNLV全收益': 'h40098.SH', '380SNLV全收益': 'h40099.SH'}
"""

{'50等权': '000050.SH',
 '180等权': '000051.SH',
 '50基本': '000052.SH',
 '180基本': '000053.SH',
 '能源等权': '000070.SH',
 '材料等权': '000071.SH',
 '工业等权': '000072.SH',
 '可选等权': '000073.SH',
 '消费等权': '000074.SH',
 '医药等权': '000075.SH',
 '金融等权': '000076.SH',
 '信息等权': '000077.SH',
 '电信等权': '000078.SH',
 '公用等权': '000079.SH',
 '180分层': '000093.SH',
 '上证F200': '000098.SH',
 '上证F300': '000099.SH',
 '上证F500': '000100.SH',
 '380等权': '000115.SH',
 '100等权': '000124.SH',
 '380基本': '000128.SH',
 '180波动': '000129.SH',
 '380波动': '000130.SH',
 '上证180高贝塔': '000135.SH',
 '上证180低贝塔': '000136.SH',
 '上证380高贝塔': '000137.SH',
 '上证380低贝塔': '000138.SH',
 '50AH优选': '000170.SH',
 '300波动(SH)': '000803.SH',
 '500波动(SH)': '000804.SH',
 '沪深300高贝塔': '000828.SH',
 '沪深300低贝塔(SH)': '000829.SH',
 '中证500高贝塔(SH)': '000830.SH',
 '中证500低贝塔(SH)': '000831.SH',
 '中证800等权': '000842.SH',
 '沪深300动态(SH)': '000843.SH',
 '沪深300稳定(SH)': '000844.SH',
 '百发100': '000851.SH',
 '基本200(SH)': '000965.SH',
 '基本400': '000966.SH',
 '基本600(SH)': '000967.SH',

In [57]:
df = pro.index_weight(index_code='000828.SH', start_date='20210901', end_date='20211023')
df.con_code.unique().tolist()


[]

In [169]:
def get_sw_codes():        
    df_stock= pro.index_member(index_code=self.index_code)
    self.stock_list = df_stock.con_code.to_list()
    return self.stock_list
#获取申万二级行业列表
pro = ts.pro_api()
df = pro.index_classify(level='L3', src='SW2021')

In [170]:
"""
#申万二级行业指数
{'种植业': '801016.SI', '渔业': '801015.SI', '林业Ⅱ': '801011.SI', '饲料': '801014.SI', '农产品加工': '801012.SI', '养殖业': '801017.SI',
 '动物保健Ⅱ': '801018.SI', '农业综合Ⅱ': '801019.SI', '化学原料': '801033.SI', '化学制品': '801034.SI', '化学纤维': '801032.SI', '塑料': '801036.SI',
 '橡胶': '801037.SI', '农化制品': '801038.SI', '非金属材料Ⅱ': '801039.SI', '冶钢原料': '801043.SI', '普钢': '801044.SI', '特钢Ⅱ': '801045.SI',
 '金属新材料': '801051.SI', '工业金属': '801055.SI', '贵金属': '801053.SI', '小金属': '801054.SI', '能源金属': '801056.SI', '半导体': '801081.SI',
 '元件': '801083.SI', '光学光电子': '801084.SI', '其他电子Ⅱ': '801082.SI', '消费电子': '801085.SI', '电子化学品Ⅱ': '801086.SI', '汽车零部件': '801093.SI',
 '汽车服务': '801092.SI', '摩托车及其他': '801881.SI', '乘用车': '801095.SI', '商用车': '801096.SI', '白色家电': '801111.SI', '黑色家电': '801112.SI',
 '小家电': '801113.SI', '厨卫电器': '801114.SI', '照明设备Ⅱ': '801115.SI', '家电零部件Ⅱ': '801116.SI', '其他家电Ⅱ': '801117.SI', '食品加工': '801124.SI',
 '白酒Ⅱ': '801125.SI', '非白酒': '801126.SI', '饮料乳品': '801127.SI', '休闲食品': '801128.SI', '调味发酵品Ⅱ': '801129.SI', '纺织制造': '801131.SI',
 '服装家纺': '801132.SI', '饰品': '801133.SI', '造纸': '801143.SI', '包装印刷': '801141.SI', '家居用品': '801142.SI', '文娱用品': '801145.SI',
 '化学制药': '801151.SI', '中药Ⅱ': '801155.SI', '生物制品': '801152.SI', '医药商业': '801154.SI', '医疗器械': '801153.SI', '医疗服务': '801156.SI',
 '电力': '801161.SI', '燃气Ⅱ': '801163.SI', '物流': '801178.SI', '铁路公路': '801179.SI', '航空机场': '801991.SI', '航运港口': '801992.SI',
 '房地产开发': '801181.SI', '房地产服务': '801183.SI', '贸易Ⅱ': '801202.SI', '一般零售': '801203.SI', '专业连锁Ⅱ': '801204.SI', '互联网电商': '801206.SI',
 '旅游零售Ⅱ': '801207.SI', '体育Ⅱ': '801216.SI', '本地生活服务Ⅱ': '801217.SI', '专业服务': '801218.SI', '酒店餐饮': '801219.SI', '旅游及景区': '801993.SI',
 '教育': '801994.SI', '国有大型银行Ⅱ': '801782.SI', '股份制银行Ⅱ': '801783.SI', '城商行Ⅱ': '801784.SI', '农商行Ⅱ': '801785.SI', '其他银行Ⅱ': '801786.SI',
 '证券Ⅱ': '801193.SI', '保险Ⅱ': '801194.SI', '多元金融': '801191.SI', '综合Ⅱ': '801231.SI', '水泥': '801711.SI', '玻璃玻纤': '801712.SI',
 '装修建材': '801713.SI', '房屋建设Ⅱ': '801721.SI', '装修装饰Ⅱ': '801722.SI', '基础建设': '801723.SI', '专业工程': '801724.SI', '工程咨询服务Ⅱ': '801726.SI',
 '电机Ⅱ': '801731.SI', '其他电源设备Ⅱ': '801733.SI', '光伏设备': '801735.SI', '风电设备': '801736.SI', '电池': '801737.SI', '电网设备': '801738.SI',
 '通用设备': '801072.SI', '专用设备': '801074.SI', '轨交设备Ⅱ': '801076.SI', '工程机械': '801077.SI', '自动化设备': '801078.SI', '航天装备Ⅱ': '801741.SI',
 '航空装备Ⅱ': '801742.SI', '地面兵装Ⅱ': '801743.SI', '航海装备Ⅱ': '801744.SI', '军工电子Ⅱ': '801745.SI', '计算机设备': '801101.SI', 'IT服务Ⅱ': '801103.SI',
 '软件开发': '801104.SI', '游戏Ⅱ': '801764.SI', '广告营销': '801765.SI', '影视院线': '801766.SI', '数字媒体': '801767.SI', '社交Ⅱ': '801768.SI',
 '出版': '801769.SI', '电视广播Ⅱ': '801995.SI', '通信服务': '801223.SI', '通信设备': '801102.SI', '煤炭开采': '801951.SI', '焦炭Ⅱ': '801952.SI',
 '油气开采Ⅱ': '801961.SI', '油服工程': '801962.SI', '炼化及贸易': '801963.SI', '环境治理': '801971.SI', '环保设备Ⅱ': '801972.SI', '个护用品': '801981.SI',
 '化妆品': '801982.SI',
 '医疗美容': '801983.SI'}
"""
"""
#申万一级行业
{'农林牧渔': '801010.SI', '基础化工': '801030.SI', '钢铁': '801040.SI', '有色金属': '801050.SI', '电子': '801080.SI', '汽车': '801880.SI',
 '家用电器': '801110.SI', '食品饮料': '801120.SI', '纺织服饰': '801130.SI', '轻工制造': '801140.SI', '医药生物': '801150.SI', '公用事业': '801160.SI',
 '交通运输': '801170.SI', '房地产': '801180.SI', '商贸零售': '801200.SI', '社会服务': '801210.SI', '银行': '801780.SI', '非银金融': '801790.SI',
 '综合': '801230.SI', '建筑材料': '801710.SI', '建筑装饰': '801720.SI', '电力设备': '801730.SI', '机械设备': '801890.SI', '国防军工': '801740.SI',
 '计算机': '801750.SI', '传媒': '801760.SI', '通信': '801770.SI', '煤炭': '801950.SI', '石油石化': '801960.SI', '环保': '801970.SI', '美容护理': '801980.SI'}

"""

"""
申万三级行业
{'种子': '850111.SI', '粮食种植': '850112.SI', '其他种植业': '850113.SI', '食用菌': '850114.SI', '海洋捕捞': '850121.SI', '水产养殖': '850122.SI',
 '林业Ⅲ': '850131.SI', '畜禽饲料': '850142.SI', '水产饲料': '850143.SI', '宠物食品': '850144.SI', '果蔬加工': '850151.SI', '粮油加工': '850152.SI',
 '其他农产品加工': '850154.SI', '生猪养殖': '850172.SI', '肉鸡养殖': '850173.SI', '其他养殖': '850174.SI', '动物保健Ⅲ': '850181.SI', '农业综合Ⅲ': '850191.SI',
 '纯碱': '850321.SI', '氯碱': '850322.SI', '无机盐': '850323.SI', '其他化学原料': '850324.SI', '煤化工': '850325.SI', '钛白粉': '850326.SI',
 '涂料油墨': '850335.SI', '民爆制品': '850337.SI', '纺织化学制品': '850338.SI', '其他化学制品': '850339.SI', '氟化工': '850382.SI', '聚氨酯': '850372.SI',
 '食品及饲料添加剂': '850135.SI', '有机硅': '850136.SI', '胶黏剂及胶带': '850137.SI', '涤纶': '850341.SI', '粘胶': '850343.SI', '其他化学纤维': '850344.SI',
 '氨纶': '850345.SI', '锦纶': '850346.SI', '其他塑料制品': '850351.SI', '改性塑料': '850353.SI', '合成树脂': '850354.SI', '膜材料': '850355.SI',
 '其他橡胶制品': '850362.SI', '炭黑': '850363.SI', '橡胶助剂': '850364.SI', '氮肥': '850331.SI', '磷肥及磷化工': '850332.SI',
 '农药': '850333.SI', '钾肥': '850336.SI', '复合肥': '850381.SI', '非金属材料Ⅲ': '850523.SI', '铁矿石': '850431.SI', '冶钢辅料': '850432.SI',
 '长材': '850441.SI', '板材': '850442.SI', '钢铁管材': '850443.SI', '特钢Ⅲ': '850401.SI', '其他金属新材料': '850521.SI', '磁性材料': '850522.SI',
 '铝': '850551.SI', '铜': '850552.SI', '铅锌': '850553.SI', '黄金': '850531.SI', '白银': '850532.SI', '稀土': '850541.SI', '钨': '850542.SI',
 '其他小金属': '850544.SI', '钼': '850545.SI', '钴': '850561.SI', '镍': '850562.SI', '锂': '850543.SI', '分立器件': '850812.SI', '半导体材料': '850813.SI',
 '数字芯片设计': '850814.SI', '模拟芯片设计': '850815.SI', '集成电路制造': '850816.SI', '集成电路封测': '850817.SI', '半导体设备': '850818.SI',
 '印制电路板': '850822.SI', '被动元件': '850823.SI', '面板': '850831.SI', 'LED': '850832.SI', '光学元件': '850833.SI', '其他电子Ⅲ': '850841.SI',
 '品牌消费电子': '850853.SI', '消费电子零部件及组装': '850854.SI', '电子化学品Ⅲ': '850861.SI', '车身附件及饰件': '850922.SI', '底盘与发动机系统': '850923.SI',
 '轮胎轮毂': '850924.SI', '其他汽车零部件': '850925.SI', '汽车电子电气系统': '850926.SI', '汽车经销商': '850232.SI', '汽车综合服务': '850233.SI',
 '其他运输设备': '858811.SI', '摩托车': '858812.SI', '电动乘用车': '850951.SI', '综合乘用车': '850952.SI', '商用载货车': '850912.SI', '商用载客车': '850913.SI',
 '空调': '851112.SI', '冰洗': '851116.SI', '彩电': '851121.SI', '其他黑色家电': '851122.SI', '厨房小家电': '851131.SI', '清洁小家电': '851132.SI',
 '个护小家电': '851133.SI', '厨房电器': '851141.SI', '卫浴电器': '851142.SI', '照明设备Ⅲ': '851151.SI', '家电零部件Ⅲ': '851161.SI', '其他家电Ⅲ': '851171.SI',
 '肉制品': '851241.SI', '其他食品': '851244.SI', '预加工食品': '851246.SI', '保健品': '851247.SI', '白酒Ⅲ': '851251.SI', '啤酒': '851232.SI', '其他酒类': '851233.SI',
 '软饮料': '851271.SI', '乳品': '851243.SI', '零食': '851281.SI', '烘焙食品': '851282.SI', '熟食': '851283.SI', '调味发酵品Ⅲ': '851242.SI', '棉纺': '851312.SI',
 '印染': '851314.SI', '辅料': '851315.SI', '其他纺织': '851316.SI', '纺织鞋类制造': '851317.SI', '鞋帽及其他': '851325.SI', '家纺': '851326.SI',
 '运动服装': '851328.SI', '非运动服装': '851329.SI', '钟表珠宝': '851331.SI', '多品类奢侈品': '851332.SI', '其他饰品': '851333.SI', '大宗用纸': '851412.SI',
 '特种纸': '851413.SI', '印刷': '851422.SI', '金属包装': '851423.SI', '塑料包装': '851424.SI', '纸包装': '851425.SI', '综合包装': '851426.SI',
 '瓷砖地板': '851436.SI', '成品家居': '851437.SI', '定制家居': '851438.SI', '卫浴制品': '851439.SI', '其他家居用品': '851491.SI', '文化用品': '851451.SI',
 '娱乐用品': '851452.SI', '原料药': '851511.SI', '化学制剂': '851512.SI', '中药Ⅲ': '851521.SI', '血液制品': '851522.SI', '疫苗': '851523.SI',
 '其他生物制品': '851524.SI', '医药流通': '851542.SI', '线下药店': '851543.SI', '互联网药店': '851544.SI', '医疗设备': '851532.SI', '医疗耗材': '851533.SI',
 '体外诊断': '851534.SI', '诊断服务': '851562.SI', '医疗研发外包': '851563.SI', '医院': '851564.SI', '其他医疗服务': '851565.SI', '火力发电': '851611.SI',
 '水力发电': '851612.SI', '热力服务': '851614.SI', '光伏发电': '851616.SI', '风力发电': '851617.SI', '核力发电': '851618.SI', '其他能源发电': '851619.SI',
 '电能综合服务': '851610.SI', '燃气Ⅲ': '851631.SI', '原材料供应链服务': '851782.SI', '中间产品及消费品供应链服务': '851783.SI', '快递': '851784.SI',
 '跨境物流': '851785.SI', '仓储物流': '851786.SI', '公路货运': '851787.SI', '高速公路': '851731.SI', '公交': '851721.SI', '铁路运输': '851771.SI',
 '航空运输': '851741.SI', '机场': '851751.SI', '航运': '851761.SI', '港口': '851711.SI', '住宅开发': '851811.SI', '商业地产': '851812.SI',
 '产业地产': '851813.SI', '物业管理': '851831.SI', '房产租赁经纪': '851832.SI', '房地产综合服务': '851833.SI', '贸易Ⅲ': '852021.SI',
 '百货': '852031.SI', '超市': '852032.SI', '多业态零售': '852033.SI', '商业物业经营': '852034.SI', '专业连锁Ⅲ': '852041.SI', '综合电商': '852061.SI',
 '跨境电商': '852062.SI', '电商服务': '852063.SI', '旅游零售Ⅲ': '852071.SI', '体育Ⅲ': '852161.SI', '本地生活服务Ⅲ': '852171.SI',
 '人力资源服务': '852181.SI', '检测服务': '852182.SI', '会展服务': '852183.SI', '其他专业服务': '852184.SI', '酒店': '852121.SI',
 '餐饮': '852141.SI', '博彩': '859931.SI', '人工景区': '852111.SI', '自然景区': '852112.SI', '旅游综合': '852131.SI', '学历教育': '859851.SI',
 '培训教育': '859852.SI', '教育运营及其他': '859853.SI', '国有大型银行Ⅲ': '857821.SI', '股份制银行Ⅲ': '857831.SI', '城商行Ⅲ': '857841.SI',
 '农商行Ⅲ': '857851.SI', '其他银行Ⅲ': '857861.SI', '证券Ⅲ': '851931.SI', '保险Ⅲ': '851941.SI', '金融控股': '851922.SI', '期货': '851923.SI',
 '信托': '851924.SI', '租赁': '851925.SI', '金融信息服务': '851926.SI', '资产管理': '851927.SI', '其他多元金融': '851928.SI', '综合Ⅲ': '852311.SI',
 '水泥制造': '857111.SI', '水泥制品': '857112.SI', '玻璃制造': '857121.SI', '玻纤制造': '857122.SI', '耐火材料': '850615.SI', '管材': '850616.SI',
 '其他建材': '850614.SI', '防水材料': '850617.SI', '涂料': '850618.SI', '房屋建设Ⅲ': '850623.SI', '装修装饰Ⅲ': '857221.SI', '基建市政工程': '857236.SI',
 '园林工程': '857251.SI', '钢结构': '857241.SI', '化学工程': '857242.SI', '国际工程': '857243.SI', '其他专业工程': '857244.SI', '工程咨询服务Ⅲ': '857261.SI',
 '电机Ⅲ': '850741.SI', '综合电力设备商': '857331.SI', '火电设备': '857334.SI', '其他电源设备Ⅲ': '857336.SI', '硅料硅片': '857351.SI',
 '光伏电池组件': '857352.SI', '逆变器': '857353.SI', '光伏辅材': '857354.SI', '光伏加工设备': '857355.SI', '风电整机': '857361.SI',
 '风电零部件': '857362.SI', '锂电池': '857371.SI', '电池化学品': '857372.SI', '锂电专用设备': '857373.SI', '燃料电池': '857374.SI',
 '蓄电池及其他电池': '857375.SI', '输变电设备': '857381.SI', '配电设备': '857382.SI', '电网自动化设备': '857321.SI', '电工仪器仪表': '857323.SI',
 '线缆部件及其他': '857344.SI', '机床工具': '850711.SI', '磨具磨料': '850713.SI', '制冷空调设备': '850715.SI', '其他通用设备': '850716.SI',
 '仪器仪表': '850731.SI', '金属制品': '850751.SI', '能源及重型设备': '850725.SI', '楼宇设备': '850728.SI', '纺织服装设备': '850721.SI',
 '农用机械': '850723.SI', '印刷包装机械': '850726.SI', '其他专用设备': '850727.SI', '轨交设备Ⅲ': '850936.SI', '工程机械整机': '850771.SI',
 '工程机械器件': '850772.SI', '机器人': '850781.SI', '工控设备': '850782.SI', '激光设备': '850783.SI', '其他自动化设备': '850784.SI', '航天装备Ⅲ': '857411.SI',
 '航空装备Ⅲ': '857421.SI', '地面兵装Ⅲ': '857431.SI', '航海装备Ⅲ': '850935.SI', '军工电子Ⅲ': '857451.SI', '安防设备': '850702.SI',
 '其他计算机设备': '850703.SI', 'IT服务Ⅲ': '852226.SI', '垂直应用软件': '851041.SI', '横向通用软件': '851042.SI', '游戏Ⅲ': '857641.SI',
 '营销代理': '857651.SI', '广告媒体': '857652.SI', '影视动漫制作': '857661.SI', '院线': '857662.SI', '视频媒体': '857671.SI', '音频媒体': '857672.SI',
 '图片媒体': '857673.SI', '门户网站': '857674.SI', '文字媒体': '857675.SI', '其他数字媒体': '857676.SI', '社交Ⅲ': '857681.SI', '教育出版': '857691.SI',
 '大众出版': '857692.SI', '其他出版': '857693.SI', '电视广播Ⅲ': '859951.SI', '电信运营商': '852212.SI', '通信工程及服务': '852213.SI',
 '通信应用增值服务': '852214.SI', '通信网络设备及器件': '851024.SI', '通信线缆及配套': '851025.SI', '通信终端及配件': '851026.SI', '其他通信设备': '851027.SI',
 '动力煤': '859511.SI', '焦煤': '859512.SI', '焦炭Ⅲ': '859521.SI', '油气开采Ⅲ': '859611.SI', '油田服务': '859621.SI', '油气及炼化工程': '859622.SI',
 '炼油化工': '859631.SI', '油品石化贸易': '859632.SI', '其他石化': '859633.SI', '大气治理': '859711.SI', '水务及水治理': '859712.SI', '固废治理': '859713.SI',
 '综合环境治理': '859714.SI', '环保设备Ⅲ': '859721.SI', '生活用纸': '859811.SI', '洗护用品': '859812.SI', '化妆品制造及其他': '859821.SI', '品牌化妆品': '859822.SI',
 '医美耗材': '859831.SI',
 '医美服务': '859832.SI'}
"""

df
code=df.index_code.values
name=df.industry_name.values
dict(zip(name,code))

{'种子': '850111.SI',
 '粮食种植': '850112.SI',
 '其他种植业': '850113.SI',
 '食用菌': '850114.SI',
 '海洋捕捞': '850121.SI',
 '水产养殖': '850122.SI',
 '林业Ⅲ': '850131.SI',
 '畜禽饲料': '850142.SI',
 '水产饲料': '850143.SI',
 '宠物食品': '850144.SI',
 '果蔬加工': '850151.SI',
 '粮油加工': '850152.SI',
 '其他农产品加工': '850154.SI',
 '生猪养殖': '850172.SI',
 '肉鸡养殖': '850173.SI',
 '其他养殖': '850174.SI',
 '动物保健Ⅲ': '850181.SI',
 '农业综合Ⅲ': '850191.SI',
 '纯碱': '850321.SI',
 '氯碱': '850322.SI',
 '无机盐': '850323.SI',
 '其他化学原料': '850324.SI',
 '煤化工': '850325.SI',
 '钛白粉': '850326.SI',
 '涂料油墨': '850335.SI',
 '民爆制品': '850337.SI',
 '纺织化学制品': '850338.SI',
 '其他化学制品': '850339.SI',
 '氟化工': '850382.SI',
 '聚氨酯': '850372.SI',
 '食品及饲料添加剂': '850135.SI',
 '有机硅': '850136.SI',
 '胶黏剂及胶带': '850137.SI',
 '涤纶': '850341.SI',
 '粘胶': '850343.SI',
 '其他化学纤维': '850344.SI',
 '氨纶': '850345.SI',
 '锦纶': '850346.SI',
 '其他塑料制品': '850351.SI',
 '改性塑料': '850353.SI',
 '合成树脂': '850354.SI',
 '膜材料': '850355.SI',
 '其他橡胶制品': '850362.SI',
 '炭黑': '850363.SI',
 '橡胶助剂': '850364.SI',
 '氮肥': '850